## Import

In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np

## Data Load

In [ ]:
day_info = pd.read_csv("../DAT/day_info.csv")
train = pd.read_csv("../DAT/train.csv")

In [ ]:
train

,ID,제품,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,B002-00003-00001,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,B002-03799-00002,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15886,15886,B002-03799-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,...,0,0,0,3,0,2,4,1,1,3
15887,15887,B002-03799-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15888,15888,B002-03799-00005,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
day_info

,Unnamed: 0,Year,Month,Day,Weekday,Weekday_Name,주말여부,공휴일여부,휴일여부
0,2022-01-01,2022,1,1,5,토,Y,Y,Y
1,2022-01-02,2022,1,2,6,일,Y,N,Y
2,2022-01-03,2022,1,3,0,월,N,N,N
3,2022-01-04,2022,1,4,1,화,N,N,N
4,2022-01-05,2022,1,5,2,수,N,N,N
...,...,...,...,...,...,...,...,...,...
454,2023-03-31,2023,3,31,4,금,N,N,N
455,2023-04-01,2023,4,1,5,토,Y,N,Y
456,2023-04-02,2023,4,2,6,일,Y,N,Y
457,2023-04-03,2023,4,3,0,월,N,N,N


## Make Dataset

In [ ]:
#필요없는 칼럼 제거와 판매량이 너무 많거나 너무 적은 날 제거

lst = train.columns[6:]
date = []

for i in lst:
    if (train[i].sum() < 100000) | (train[i].sum() > 700000) :
        date.append(i)
        train = train.drop(i, axis=1)
day_info = day_info.drop(day_info[day_info["Unnamed: 0"].isin(date)].index)

In [ ]:
values_df = pd.concat([train.iloc[:,0], train.iloc[:,6:]], axis = 1)
values_df = values_df.melt(id_vars = 'ID', var_name = 'YEAR', value_name = 'COUNT')
values_df = values_df.iloc[:, [0,2,1]]
values_df

,ID,COUNT,YEAR
0,0,0,2022-01-01
1,1,0,2022-01-01
2,2,0,2022-01-01
3,3,0,2022-01-01
4,4,0,2022-01-01
...,...,...,...
6721465,15885,0,2023-04-04
6721466,15886,3,2023-04-04
6721467,15887,0,2023-04-04
6721468,15888,2,2023-04-04


In [ ]:
tr_df = values_df
tr_df = tr_df.iloc[:, [0,1,2]]
tr_df = pd.merge(tr_df, train.iloc[:, [0, 1,2,3,4,5]], how = 'left', on = 'ID')
tr_df

,ID,COUNT,YEAR,제품,대분류,중분류,소분류,브랜드
0,0,0,2022-01-01,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001
1,1,0,2022-01-01,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002
2,2,0,2022-01-01,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002
3,3,0,2022-01-01,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002
4,4,0,2022-01-01,B002-00003-00001,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003
...,...,...,...,...,...,...,...,...
6721465,15885,0,2023-04-04,B002-03799-00002,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799
6721466,15886,3,2023-04-04,B002-03799-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799
6721467,15887,0,2023-04-04,B002-03799-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799
6721468,15888,2,2023-04-04,B002-03799-00005,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799


In [ ]:
tr_df = pd.concat([tr_df, pd.DataFrame(np.repeat(day_info.values, 15890, axis=0), columns = day_info.columns)], axis = 1)
tr_df

,ID,COUNT,YEAR,제품,대분류,중분류,소분류,브랜드,Unnamed: 0,Year,Month,Day,Weekday,Weekday_Name,주말여부,공휴일여부,휴일여부
0,0,0,2022-01-01,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022-01-01,2022,1,1,5,토,Y,Y,Y
1,1,0,2022-01-01,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2022-01-01,2022,1,1,5,토,Y,Y,Y
2,2,0,2022-01-01,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2022-01-01,2022,1,1,5,토,Y,Y,Y
3,3,0,2022-01-01,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2022-01-01,2022,1,1,5,토,Y,Y,Y
4,4,0,2022-01-01,B002-00003-00001,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,2022-01-01,2022,1,1,5,토,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6721465,15885,0,2023-04-04,B002-03799-00002,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,2023-04-04,2023,4,4,1,화,N,N,N
6721466,15886,3,2023-04-04,B002-03799-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,2023-04-04,2023,4,4,1,화,N,N,N
6721467,15887,0,2023-04-04,B002-03799-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,2023-04-04,2023,4,4,1,화,N,N,N
6721468,15888,2,2023-04-04,B002-03799-00005,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,2023-04-04,2023,4,4,1,화,N,N,N


In [ ]:
tr_df = tr_df.drop(["Unnamed: 0", "Weekday_Name", "YEAR"], axis=1)
tr_df = tr_df.sort_values(by=['ID', 'Year', 'Month', 'Day'])
tr_df.reset_index(inplace=True, drop=True)
tr_df

,ID,COUNT,제품,대분류,중분류,소분류,브랜드,Year,Month,Day,Weekday,주말여부,공휴일여부,휴일여부
0,0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022,1,1,5,Y,Y,Y
1,0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022,1,2,6,Y,N,Y
2,0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022,1,3,0,N,N,N
3,0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022,1,4,1,N,N,N
4,0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022,1,5,2,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6721465,15889,0,B002-03799-00010,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,2023,3,31,4,N,N,N
6721466,15889,0,B002-03799-00010,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,2023,4,1,5,Y,N,Y
6721467,15889,0,B002-03799-00010,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,2023,4,2,6,Y,N,Y
6721468,15889,0,B002-03799-00010,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,2023,4,3,0,N,N,N


In [ ]:
window_size = 60
lag_range = range(window_size)

# Prepare a list to hold DataFrames for each ID
id_dataframes = []

for id_idx in tqdm(range(0, 15890)):
    id_window = tr_df[tr_df['ID'] == id_idx]  # Filter data for the current ID
    id_counts = id_window['COUNT'].reset_index(drop=True)  # Extract COUNT column
    id_lagged_columns = {f'COUNTS_{lag}': id_counts.shift(lag) for lag in lag_range}
    id_df = pd.DataFrame(id_lagged_columns)
    id_df['ID'] = id_idx
    id_dataframes.append(id_df)

# Concatenate all ID DataFrames into a single DataFrame
counts = pd.concat(id_dataframes, ignore_index=True)

100%|██████████| 15890/15890 [03:51<00:00, 68.77it/s]


In [ ]:
counts

,COUNTS_0,COUNTS_1,COUNTS_2,COUNTS_3,COUNTS_4,COUNTS_5,COUNTS_6,COUNTS_7,COUNTS_8,COUNTS_9,...,COUNTS_51,COUNTS_52,COUNTS_53,COUNTS_54,COUNTS_55,COUNTS_56,COUNTS_57,COUNTS_58,COUNTS_59,ID
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6721465,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15889
6721466,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15889
6721467,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15889
6721468,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15889


In [ ]:
tr_df = pd.concat([tr_df, counts], axis=1)
tr_df.dropna(inplace=True, axis = 0)
tr_df.reset_index(drop=True, inplace=True)
tr_df

NameError: ignored

In [ ]:
tr_df = tr_df.iloc[:, :-1] #맨 뒤에 생긴 ID칼럼 제거
tr_df

,ID,COUNT,제품,대분류,중분류,소분류,브랜드,Year,Month,Day,...,COUNTS_50,COUNTS_51,COUNTS_52,COUNTS_53,COUNTS_54,COUNTS_55,COUNTS_56,COUNTS_57,COUNTS_58,COUNTS_59
0,0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022,3,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022,3,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022,3,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022,3,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783955,15889,0,B002-03799-00010,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,2023,3,31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5783956,15889,0,B002-03799-00010,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,2023,4,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5783957,15889,0,B002-03799-00010,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,2023,4,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5783958,15889,0,B002-03799-00010,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,2023,4,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Save

In [ ]:
tr_df.to_csv('../OUT/melt_data60.csv', index=False)